In [63]:
!pip install cryptography

# **Utlity Functions**

In [64]:
# Convert to string in ascii from numeric representation of triplets
def convert(a) :
    l = len(a)
    s = ""
    
    idx = 0
    while idx < l :
        temp = a[idx:idx+3]
        s = s + chr(int(temp))
        idx = idx + 3
        
    return s

# Creating a random salt of length n
import string
import random

def get_random_salt(n) :
  
    salt=""
    for i in range(n):
        salt=salt+random.choice(string.ascii_letters)

    return salt

# Get a random encryption key
from cryptography.fernet import Fernet

def get_random_encryptkey() :

    encrypt_key = Fernet.generate_key()
    # fernet = Fernet(key)
    return encrypt_key


In [65]:
# Take XOR of string a and string b
def XOR1(a, b) :
    res = ""
    i = 0
    j = 0
    
    while (i<len(a) and j<len(b)) :
        
        temp = str(ord(a[i]) ^ ord(b[j])) 
        
        if len(temp) == 1 :
            temp = "00" + temp
        elif len(temp) == 2 :
            temp = "0" + temp
        
        res += temp
        i = i + 1
        j = j + 1
    
    while i<len(a) :
        
        temp = str(ord(a[i])) 
        
        if len(temp) == 1 :
            temp = "00" + temp
        elif len(temp) == 2 :
            temp = "0" + temp
        
        res += temp
        i = i + 1
        
    while j<len(b) :
        
        temp = str(ord(b[j])) 
        
        if len(temp) == 1 :
            temp = "00" + temp
        elif len(temp) == 2 :
            temp = "0" + temp
        
        res += temp
        j = j + 1
        
    return res      


# **Connecting with MySQL**

In [66]:
pip install mysql-connector-python-rf

In [67]:
# install, set connection
!apt-get install mysql-server > /dev/null
!service mysql start
!mysql -e "ALTER USER 'root'@'localhost' IDENTIFIED WITH mysql_native_password BY 'root'"
!pip -q install PyMySQL
%load_ext sql
%config SqlMagic.feedback=False 
%config SqlMagic.autopandas=True
%sql mysql+pymysql://root:root@/
# query using %sql or %%sql
df = %sql SELECT Host, User, authentication_string FROM mysql.user
df

 * Starting MySQL database server mysqld
   ...done.
ERROR 1045 (28000): Access denied for user 'root'@'localhost' (using password: NO)
The sql extension is already loaded. To reload it, use:
  %reload_ext sql
 * mysql+pymysql://root:***@/


,Host,User,authentication_string
0,localhost,root,*81F5E21E35407D884A6CD4A731AEBFB6AF209E1B
1,localhost,mysql.session,*THISISNOTAVALIDPASSWORDTHATCANBEUSEDHERE
2,localhost,mysql.sys,*THISISNOTAVALIDPASSWORDTHATCANBEUSEDHERE
3,localhost,debian-sys-maint,*53E101956E13ED24AC7FD125DEBDFF39E932C594


In [68]:
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="root"
)

print(mydb)

# **Part 1 : Store Data in Database**

In [69]:
mycursor = mydb.cursor()

In [70]:
mycursor.execute("DROP DATABASE btp")

DatabaseError: ignored

In [71]:
mycursor.execute("CREATE DATABASE btp")

In [72]:
mycursor.execute("SHOW DATABASES")
for x in mycursor:
  print(x)

('information_schema',)
('btp',)
('mysql',)
('performance_schema',)
('sys',)


In [73]:
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="root",
  database='btp'
)

mycursor = mydb.cursor()

In [74]:
table_name = "customers"
columns = ['id', 'name', 'city', 'wallet_balance']
data = [ ('1', 'Mangesh', 'Umarkhed', '100'),
        ('2', 'Puspesh', 'Patna', '200'),
        ('3', 'Ritwiz', 'Delhi', '150'),
        ('4', 'Pranay', 'Delhi', '190'),
        ('5', 'Harsh', 'Agra', '120') ]

In [75]:
def store_data(table_name, columns, data) :

    # create table query, first column shoud be id/primary key
    create_query = "CREATE TABLE " + table_name + " (" + columns[0] + " VARCHAR(511) PRIMARY KEY"

    for idx in range(1,len(columns)):
        create_query += ", " + columns[idx] + " VARCHAR(511)"
    
    create_query += ")"

    mycursor.execute(create_query)

    # insert data query 
    insert_query = "INSERT INTO " + table_name + " (" + columns[0]

    for idx in range(1,len(columns)):
        insert_query += ", " + columns[idx]
    
    insert_query += ") VALUES (" + "%s"

    for idx in range(1,len(columns)):
        insert_query += ", %s"

    insert_query += ")"

    id_and_salts = []
    encrypt_key = get_random_encryptkey();
    fernet = Fernet(encrypt_key)
    encrypted_data = []
    
    for idx in range(len(data)):
        encrypted_data.append(list(data[idx]))

    for row in range(len(data)) :
      
        encrypted_id = fernet.encrypt(data[row][0].encode('utf-8'))
        salt = get_random_salt(len(encrypted_id))
        encrypted_salt = fernet.encrypt(salt.encode('utf-8'))
        
        id_and_salts.append(encrypted_id)
        id_and_salts.append(encrypted_salt)

        encrypted_data[row][0] = XOR1(str(encrypted_id),salt)

        for col in range(1,len(data[0])) :

            encrypted_data[row][col] = XOR1(str(fernet.encrypt(data[row][col].encode('utf-8'))), salt)
                
    mycursor.executemany(insert_query, encrypted_data)
    mydb.commit()

    print("Encryption Key :", encrypt_key)

    idx = 0
    print("Encrypted Primary Key and Salt :")
    while idx < len(id_and_salts) :
        if idx == 0:
            print(str(id_and_salts[idx]) + "," + str(id_and_salts[idx+1]))
        else :
            print("," + str(id_and_salts[idx]) + "," + str(id_and_salts[idx+1]))
        idx = idx + 2


In [76]:
store_data(table_name, columns, data)

Encryption Key : b'yER6V-sJWXVFfkAi6aPDKogoE_qep31ovb8dgL9YD0M='
Encrypted Primary Key and Salt :
b'gAAAAABicXsxBivZxGZ5VbhFdCQHqDnSHt5ICFOkHSAGxZ1LpCrZ8z5URTSIFRKc6kxcoJk_ndvNmTOtp-hlgsl-8PV_d3U4Ww==',b'gAAAAABicXsxkL-jjmDh4wAmfXlhFMZmVE2XtBemb5ui9s17vvqZP9sH_AhTdnR4oCBAjVX2A2X-l7WPbW6lRmWYYnjXvG90n0Xg-sx9TYtEVzSIj_QX-2zta-0LKWR-t1JjBZ4vRIITKHbVsHnIa-ZZV_pBU8plXv3Dz9dhROzMZcYOOxVGho_zXz7exl9JWBEJPcwD6Rhngdt6Z89BLwclYyk4uf3N5A=='
,b'gAAAAABicXsxOxz2aq-tzymgLU4sasM2sox7KrAu8kOcN9ZDr7_hthXSX8r_ddI1gpN7d0sU7EwoKd6PdkVwto3TH61yfIxFUQ==',b'gAAAAABicXsxfKpLPhXQQPTRwNaj5iGtyB7Xahv_32Wc971TydpB_-5EsDFGyCrfRrLuA3MseEmJtHFTeNqBUmaJy5LcpzddIBT_MVB_1Nqx7kdE43Y170eQWh2yuzLSvy4r4fafz-dvui4XxN6Ttb_qX1h2rpb0udTf4kggVlnaGLAujDim7KmuHWAYb7QN2uyTCfBWKwtzYIS3xVPi1jchoQFRDLSqBw=='
,b'gAAAAABicXsxkeRW5hWiQp77_M0aT3r0j9Rwks4dDPJoQZ_-_ZVe-qvGnlfE26gnDBlcWHLSrX0e9BEjQX-bPF0OoZzJV_N6sQ==',b'gAAAAABicXsxeIZcYBBJWFMI82ATTsbdMq96Rc36QIlFHbjiRVr0xzZNOnjbOmlRbl2YilZwKTvjM6zKftIqTH-52WzKFaMAH_Id_1K77NSCBtPpNYc2gDVq-T

In [ ]:
mycursor.execute("SHOW TABLES")
for x in mycursor:
  print(x)

In [ ]:
mycursor.execute("SELECT * from customers")
for x in mycursor:
  print(x)

('026082003042037035019059020048041037004017051028091069015045036107014091045027049025002060037063023010059030044047063120006023052028048008037042056037042007103008041105021011067081002051041027047043030044066019094049019023059116019034063039002060113055048053031047020021054001007048037062059020029062061061039', '026082003042037035019059020048041037004017059092095003029080001003014012000036054014121014029027103003043062022013087002121022028029003005023047052058042034035029035006021054029001016060025064012021013027032080062052027065033054001029071039006032001061114035037000050056017111056012018072002070001024061061039', '026082003042037035019059020048041037004017005006087077004015033040012000060033045037044024020044051035067000020040082039126029018028046044058034066014034009005005008003052010001051041046046019021041043048036020005014010025037029020010061017025036006046039027022012058031039055064060036058017036059024061061039', '02608200304203703501905902004804103700401708805406305706101

# **PART 2 : Making Query from MySQL**

In [84]:

# Get from result of store_data 
key1=b'yER6V-sJWXVFfkAi6aPDKogoE_qep31ovb8dgL9YD0M='
EncPrimary_key="b'gAAAAABicXsxkeRW5hWiQp77_M0aT3r0j9Rwks4dDPJoQZ_-_ZVe-qvGnlfE26gnDBlcWHLSrX0e9BEjQX-bPF0OoZzJV_N6sQ=='"
fernet1=Fernet(key1)
# Get from Blockchain
encSalt = "b'gAAAAABicXsxeIZcYBBJWFMI82ATTsbdMq96Rc36QIlFHbjiRVr0xzZNOnjbOmlRbl2YilZwKTvjM6zKftIqTH-52WzKFaMAH_Id_1K77NSCBtPpNYc2gDVq-T5Mn9xqqaSWMyV0EK2-DwyRsZMqhMcAuBea3_vpsgU81keay2mqg_yE5yaaHi4gRZOnR9cf5EnV-eFxSUBhuR9lv-ZCJLXBHfchUhX_1A=='"
salt_temp1=encSalt[1:len(encSalt)]
salt_str= fernet1.decrypt(salt_temp1.encode('utf-8')).decode()

Idd=(XOR1(salt_str,EncPrimary_key), )

query = "SELECT name from customers where id= %s"
mycursor.execute(query,Idd)
res=""
for x in mycursor:
  res=x

Encrypted_res=XOR1(convert(res[0]),salt_str)

temp=convert(Encrypted_res)
print(temp)
s1=temp[1:len(temp)]
decMessage = fernet1.decrypt(s1.encode('utf-8')).decode()

print("\n\nDecrypted string: ", decMessage)

b'gAAAAABicXsxCUmaE3Awv3s2INz4OcMfEoLnT0aYB34sl_s7mcDDS-Fk6hPgKe7mMUPpLK6cbmSL2-oWeQZdkWncts1HtrX8CA=='


Decrypted string:  Ritwiz
